# Function documentation

In [ ]:
using Oscar;
using Combinatorics;
pm = Polymake;

In [ ]:
cd("..")

## Input data

```inputData38.jl```

```S8``` is a ```Vector{Vector{Int64}}```. It is the symmetric group $\mathfrak{S}_8$, expressed as a subgroup pf $\mathfrak{S}_{56}$ via its action on $\binom{[8]}{3}$. More precisely, we identify $\binom{[8]}{3}$ with $[56] = \{1,2,\ldots,56\}$ by ordering the elements of $\binom{[8]}{3}$ lexicographically. The action of $\mathfrak{S}_8$ on $\binom{[8]}{3}$ by

\begin{equation}
\sigma \cdot \{i,j,k\} = \{\sigma(i),\sigma(j),\sigma(k)\}
\end{equation}

This induces an action of $\mathfrak{S}_8$ on $\mathfrak{S}_{56}$.

---
```vDelta38``` is a ```Matrix{Int64}``` of dimensions $56\times 9$. Its rows are the vertices of $\Delta(3,8)$ following the "homogeneous" convention in ```polymake```. The rows are indicator vectors of the elements of $\binom{[8]}{3}$ (and starting with a $1$), and ordered lexicographically.

---
```rays``` is a ```Matrix{Int64}``` of dimensions $12\times 56$. Each row is  a $\mathfrak{S}_8$-orbit representative of a ray of $\operatorname{TGr}_0(3,8)$ with its secondary fan structure. 

---
```lin``` is a ```Matrix{Int64}``` of dimensions $9\times 56$. Its rows span the lineality space of $\operatorname{TGr}_0(3,8)$. 

---
```MaxCones38``` records the maximal cones of $\operatorname{TGr}_0(3,8)$ (with its secondary fan structure). It is a ```Vector{Tuple{Int64, Int64}}```. Each tuple represents a ray. In a given tuple ```(i,j)```, the first element refers to a row of ```rays```, the second is a row of ```S8```. The corresponding ray is ```rays[i,S8[j]]```, i.e., the ```i```-th row of ```rays```, whose entries are permuted by ```S8[j]```. 


In [ ]:
S8 = Array(pm.load_data("group38"));
S8 = pm.to_one_based_indexing(S8);
Delta38 = Polymake.polytope.hypersimplex(3,8);
vDelta38 = Matrix{Int64}(Delta38.VERTICES[1:56,1:9]);
lin = transpose(vDelta38);
rays = Matrix{Int64}(Polymake.load_data("GrRays38.data"));
MaxConesStr = readlines("ConesDrOfGr38.data");
MaxCones38_0index = [[Tuple([parse(Int64, s) for s in split(ss,"#")]) for ss in split(C," ")] for C in MaxConesStr];
MaxCones38 = pm.to_one_based_indexing(MaxCones38_0index);

## File Handling

Functions in ```fileHandling.jl```

---

**Function**: ```string2Int64Vector(s::String)```

*Description*: Given ```s``` a string of space-separated integers, returns these integers as a ```Vector{Int64}```

*Example*: ```string2Int64Vector("1 -2 3") = [1, -2, 3] ```

---

**Function**: ```file2SetVectors(fName::String)```

*Description*: The file named ```fName``` should be a file whose lines are space-separated lists of integers. This function returns a ```Vector{Vector{Int64}}``` whose entries are the vectors of integers corresponding to the lines of ```fName```.


*Example*: If the file ```test.dat``` is
```
1 -2 3
-4 5 -6
```
then ```file2SetVectors("test.dat") = [[1, -2, 3], [-4, 5, 6]] ```

---

**Function**: ```vec2String(v::Vector{Int64})```

*Description*: Returns a space-separated sting of the entries of the vector ```v```. 


*Example*: ```vec2String([1,-2, 3]) == "1 -2 3"```

---




In [ ]:
function string2Int64Vector(s)
    return map(i -> parse(Int64, i), split(s))
end

function file2SetVectors(fileName)
    return map(s -> string2Int64Vector(s), readlines(fileName))
end

function vec2String(v)
    vecString = map(i -> string(i), v)
    return join(vecString, " ")
end

## Coordinate rings for thin Schubert cells and their limits

Functions in `tscCoordRing.jl`

We compute the coordinate ring of the thin Schubert cell of a matroid. The main functions here are `TSC` and `limitTSC`. 

---

**Function**: `makePolyRing(d::Int64,n::Int64,F::Ring)`

*Description*: This creates a multivariate polynomial ring `R` in $d(n-d)$ variables, organized as a $d\times (n-d)$ matrix `x`. 

---
**Function**: `makeCoordinateMatrix(d::Int64, n::Int64, B::Vector{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B`  is a $d$ element subset of $\{1,\ldots,n\}$, and `R, x` is a polynomial ring whose variables are organized in a $d\times (n-d)$ matrix. This function returns a $d\times n$ matrix whose columns indexed by `B` form the identity, and the remaining ones are `x`. 

*Example*: 

```
R,x = makePolyRing(3, 8, QQ)
makeCoordinateMatrix(3, 8, [1,4,8], R, x) == [1 x_{1, 1} x_{1, 2} 0 x_{1, 3} x_{1, 4} x_{1, 5} 0; 0 x_{2, 1} x_{2, 2} 1 x_{2, 3} x_{2, 4} x_{2, 5} 0; 0 x_{3, 1} x_{3, 2} 0 x_{3, 3} x_{3, 4} x_{3, 5} 1]
```
---
**Function**: `reducedCoordinateMatrix(M::Matroid, d::Int64, n::Int64, B::Vector{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B` must be a basis of the matroid `M`. This produces the matrix `A` from `makeCoordinateMatrix`, but sets the `i, j` entry equal to 0 if the symmetric difference $B \Delta \{i,j\}$ is a nonbasis of the matroid `M`. 

*Example*:  
```
R,x = makePolyRing(3, 8, QQ)
M = matroid_from_nonbases([[1,2,6], [1,4,5], [1,7,8], [2,3,5], [2,4,8], [5,6,7], [3,6,8], [3,4,7]], 8)
reducedCoordinateMatrix(3, 8, [1,4,8], R, x) == [1 0 x_{1, 2} 0 x_{1, 3} x_{1, 4} x_{1, 5} 0; 0 x_{2, 1} x_{2, 2} 1 x_{2, 3} x_{2, 4} 0 0; 0 x_{3, 1} x_{3, 2} 0 0 x_{3, 4} x_{3, 5} 1]

```
---
**Function**: `TSC(M::Matroid, d::Int64, n::Int64, B::Vector{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B` must be a basis of the matroid `M`. This produces the ideal of the thin Schubert cell corresponding to `M`. This is the ideal of `R` generated by the maximal determinants of `A = reducedCoordinateMatrix(rank(M), length(matroid_groundset(M)), B, R, x)` whose columns are indexed by the nonbases of `M` and is saturated by the semigroup generated by the maximal determinants of `A` whose columns are indexed by bases of `M`. 

*Example*:  

```
R,x = makePolyRing(3, 8, QQ)
M = matroid_from_nonbases([[1,2,6], [1,4,5], [1,7,8], [2,3,5], [2,4,8], [5,6,7], [3,6,8], [3,4,7]], 8)
TSC(M, 3, 8, [1,4,8], R, x) =

```
---
**Function**: `projectedNonBases(M::Matroid, F::Ring, B::Vect{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B` must be a basis of the matroid `M`. This produces a list of the maximal determinants of `A = reducedCoordinateMatrix(rank(M), length(matroid_groundset(M), B, R, x)` formed by the columns corresponding to the *nonbases* of `M`. 

*Example*:  

```
R,x = makePolyRing(3, 8, QQ)
M = matroid_from_nonbases([[1,2,6], [1,4,5], [1,7,8], [2,3,5], [2,4,8], [5,6,7], [3,6,8], [3,4,7]], 8)
projectedNonBases(M, QQ, [1,4,8], R, x) == fmpq_mpoly[
x_{2, 1}*x_{3, 4} - x_{3, 1}*x_{2, 4}, 0, 
x_{2, 1}*x_{3, 2}*x_{1, 3} + x_{3, 1}*x_{1, 2}*x_{2, 3} - x_{3, 1}*x_{2, 2}*x_{1, 3}, 
x_{1, 3}*x_{2, 4}*x_{3, 5} - x_{2, 3}*x_{1, 4}*x_{3, 5} + x_{2, 3}*x_{3, 4}*x_{1, 5}, 
x_{1, 2}*x_{2, 4} - x_{2, 2}*x_{1, 4}, x_{1, 2}*x_{3, 5} - x_{3, 2}*x_{1, 5}]
```
---

**Function**: `projectedBases(M::Matroid, F::Ring, B::Vect{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B` must be a basis of the matroid `M`. This produces a list of the maximal determinants of `A = reducedCoordinateMatrix(rank(M), length(matroid_groundset(M), B, R, x)` formed by the columns corresponding to the *bases* of `M`. 

---
**Function**: `limitTSC(Ms::Vector{Matroid}, F::Ring, B::Vect{Int64}, R::GradedRing, x::fmpz_mpoly)`

*Description* Here, `B` must be a basis common to each matroid in `Ms`. Ideally, the matroids `Ms` correspond to a collection of maximal cells in a matroidal subdivision of a matroid polytope. This function computes the limit of thin Schubert cells of the corresponding diagram. 


In [ ]:
function makePolyRing(d::Int64, n::Int64, F::Ring)
    R,x = polynomial_ring(F, :"x"=>(1:d, 1:n-d))
    W = ones(Int64, d*(n-d))
    R,x = grade(R,W)
    x = reshape(x, (d,n-d))
    return R,x
end

function makeCoordinateMatrix(d::Int64, n::Int64, B::Vector{Int64}, R, x)
    
    Id = identity_matrix(R,d)
    S_id = matrix_space(R,d,n)
    S = matrix_space(R,d,n-d)
    X = S(x)
    
    X_id = S_id()
    
    idCount = 1
    varCount = 1
    
    for i in 1:n
        if i in B
            X_id[:,i] = Id[:,idCount]
            idCount += 1
        end
        if !(i in B)
            X_id[:,i] = X[:,varCount]
            varCount +=1
        end
    end
    
    return  X_id
end

function reducedCoordinateMatrix(M, d, n, B, R, x)
    
    X = makeCoordinateMatrix(d,n,B,R,x)
    nonBasesM = nonbases(M)
    coordNonBases = [ nb for nb in nonBasesM if length(symdiff(B,nb)) == 2]
    
    if length(coordNonBases) == 0
        return X
    end
    
    for nb in coordNonBases
        row_nb = setdiff(B,nb)[1]
        row_nb = length([a for a in B if a < row_nb]) + 1
        
        col_nb = setdiff(nb,B)[1]
        X[row_nb,col_nb] = 0
    end
    return X
    
end

function TSC(M, F, B, R, x)
    d = rank(M)
    n = length(matroid_groundset(M))
    Bs = bases(M)
    NBs = nonbases(M)
    #R,x = PolynomialRing(F, :"x"=>(1:d, 1:n-d))

    Mx = reducedCoordinateMatrix(M,d,n,B,R,x)
    I = ideal(unique!([det(Mx[:, nb]) for nb in NBs ]))
    toInvert = unique!([det(Mx[:, b])  for b in Bs])
    
    for s in toInvert
        I = saturation(I,ideal([s]))
    end
    
    return I
end

function projectedNonBases(M, F, B, R, x)
    d = rank(M)
    n = length(matroid_groundset(M))
    NBs = nonbases(M)
    Mx = reducedCoordinateMatrix(M,d,n,B,R,x)
    return unique!([det(Mx[:, nb]) for nb in NBs ]) 
end


function projectedBases(M, F, B, R, x)
    d = rank(M)
    n = length(matroid_groundset(M))
    Bs = bases(M)
    Mx = reducedCoordinateMatrix(M,d,n,B,R,x)
    return unique!([det(Mx[:, nb]) for nb in Bs ])
end

function limitTSC(Ms, F, B, R, x)
    d = rank(Ms[1])
    n = length(matroid_groundset(Ms[1]))

    nonBasesX = []
    basesX = []

    for M in Ms        
        append!(nonBasesX, projectedNonBases(M, F, B, R, x))
        append!(basesX, projectedBases(M, F, B, R, x))
    end
    
    I = ideal(R, unique!(nonBasesX))
    
    
    for s in basesX
        I = saturation(I,ideal([s])); 
    end
    
    return I
end

## Matroidal subdivisions

Functions in `matroidalSubd.jl`

**Function**: `plueckerList2Matroid(L::Vector{Int64}, d::Int64, n::Int64)`

*Description*: The vector `L` is a sublist of `Vector{Int64}(1:56)` corresponding to $\binom{[n]}{d}$ ordered lexicographically. It returns the matroid on $[n]$ whose bases are the bases indexed by `L`. 

*Example*: `plueckerList2Matroid([1,2,3,4,5], 2, 4)` is the $(2,4)$--matroid with bases $12,13,14,23,24$. 

---
**Function**: `subd2Matroids(subd::SubdivisionOfPoints, d::Int64, n::Int64)`

*Description*: Returns the list of matroids of maximal cells of `subd` in order.

*Example*: Consider the matroidal subdivision of $\Delta(2,4)$ induced by the vector `w = [1,0,0,0,0,0] `:
```
Delta24 = Polymake.polytope.hypersimplex(2,4);
vDelta24 = Matrix{Int64}(Delta24.VERTICES[1:6,1:5]);
subd = subdivision_of_points(vDelta24[:, 2:5], [1,0,0,0,0,0])
Ms = subd2Matroids(subd, 2, 4)
```
There are 2 matroids in `Ms`, their bases are: $13,14,23,24,34$ and $12,13,14,23,24,34$.

---
**Function**: `edgesDualGraph(subd::SubdivisionOfPoints)`

*Description*: Returns the codimension 1 cells of `subd` meeting the relative interior of the polytope being subdivided, i.e., the cells corresponding to the edges of the dual graph of the subdivision. 

*Example*: In the matroidal subdivision of $\Delta(2,4)$ induced by the vector `w = [1,0,0,0,0,0]`, there is only one such codim 1 cell:
```
Delta24 = Polymake.polytope.hypersimplex(2,4);
vDelta24 = Matrix{Int64}(Delta24.VERTICES[1:6,1:5]);
subd = subdivision_of_points(vDelta24[:, 2:5], [1,0,0,0,0,0])
edgesDualGraph(subd) == [[2,3,4,5]]
```

---
**Function**: `subd2CodimLeq1Matroids(subd::SubdivisionOfPoints, d::Int64, n::Int64)`

*Description*: Returns a pair of lists of matroids, the first list is the matroids of the vertices of the dual graph of `subd`, and the second is the list of matroids of the edges of the dual graph.

*Example*: In the matroidal subdivision of $\Delta(2,4)$ induced by the vector `w = [1,0,0,0,0,0]` the dual graph has 2 vertices and 1 edge:
```
Delta24 = Polymake.polytope.hypersimplex(2,4);
vDelta24 = Matrix{Int64}(Delta24.VERTICES[1:6,1:5]);
subd = subdivision_of_points(vDelta24[:, 2:5], [1,0,0,0,0,0])
maxMatroids, codim1Matroids = subd2CodimLeq1Matroids(subd, 2, 4)
nonbases(maxMatroids[1]) == [[1,2]]
nonbases(maxMatroids[2]) == [[3,4]]
nonbases(codim1Matroids[1]) == [[1,2],[3,4]]
```

**Function**: `subdDualGraph(subd::SubdivisionOfPoints)`

*Description*: Produces the dual graph of the subdivision `subd`. 

*Example*: In the matroidal subdivision of $\Delta(2,5)$ induced by the vector `w = [1,0,0,0,0,0,0,0,0,1]`, the dual graph is a tree on 3 vertices:
```
Delta25 = Polymake.polytope.hypersimplex(2,5);
vDelta25 = Matrix{Int64}(Delta25.VERTICES[1:10,1:6]);
subd = subdivision_of_points(vDelta25[:, 2:6], [1,0,0,0,0,0,0,0,0,1])
dg = subd2DualGraph(subd)
Graphs.nv(dg) == 3 
Graphs.ne(dg) == 2
```

---

**Function**: `subdMatroidsNonLeaves(subd::SubdivisionOfPoints, d::Int64, n::Int64)`

*Description*: returns the matroids corresponding to the non-leaf vertices of the dual graph of the subdivision `subd` of $\Delta(d,n)$. 

*Example*: In the matroidal subdivision of $\Delta(2,5)$ induced by the vector `w = [1,0,0,0,0,0,0,0,0,1]`, there is only one non-leaf vertex:
```
Delta25 = Polymake.polytope.hypersimplex(2,5);
vDelta25 = Matrix{Int64}(Delta25.VERTICES[1:10,1:6]);
subd = subdivision_of_points(vDelta25[:, 2:6], [1,0,0,0,0,0,0,0,0,1])
nonleafMatroids = subdMatroidsNonLeaves(subd,2,5)
nonbases(nonleafMatroids[1]) == [[1,2],[4,5]]
```

---

**Function**: `commonBasis(Ms::Vector{Matroid})`

*Description*: Returns a `Vector{Vector{Int64}}` recording the bases common to all matroids in `Ms`.   

*Example*: In the matroidal subdivision of $\Delta(2,5)$ induced by the vector `w = [1,0,0,0,0,0,0,0,0,1]`, there is only one non-leaf vertex:

```
Delta25 = Polymake.polytope.hypersimplex(2,5);
vDelta25 = Matrix{Int64}(Delta25.VERTICES[1:10,1:6]);
subd = subdivision_of_points(vDelta25[:, 2:6], [1,0,0,0,0,0,0,0,0,1])
Ms = subd2Matroids(subd,2,5)
commonBasis(Ms) == [[1,4],[1,5],[2,4],[2,5]]
```
---
**Function**: ```subdMatroidsLeaves(subd::SubdivisionOfPoints, d:: Int64, n::Int64)```

*Description*: Returns the matroids corresponding to the leaf vertices of the dual graph of ```subd```.

*Example*: 

---
**Function**: ```leavesGraph(graphOscar::graph)```

*Description*: Returns the leaf vertices of the graph.

*Example*: 

---
**Function**: ```leafEdgesDualGraphAsCells(subd::SubdivisionOfPoints)```

*Description*: Returns the cells corresponding to the edges of leaves of the dual graph of ```subd```, as a ```Vector{Vector{Int64}}```. Each cell is recorded as a vector of its vertex-indices of the ambient polytope. 

*Example*: 

---
**Function**: ```subd2LeavesAndCenter(subd::SubdivisionOfPoints, d::Int64, n::Int64)```

*Description*: Returns a triple ```leafMaxMatroids, leafEdgeMaxMatroids, remainingMaxMatroids``` where each is a list of matroids. ```leafMaxMatroids``` has the matroids of the vertices of the leaves, ```leafEdgeMaxMatroids``` has the matroids of the edges of the leaves, and ```remainingMaxMatroids``` has the matroids of the remaining vertices. 

*Example*: 



In [ ]:
function plueckerList2Matroid(L, d, n)
    dn = collect(powerset([1:n;], d, d))
    return matroid_from_bases(dn[L], 1:n) 
end

function subd2Matroids(subd, d, n)
    return [plueckerList2Matroid(r, d, n) for r in maximal_cells(subd) ]     
end

function subdDualGraph(subd)
    Gra = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    E = pm.graph.edges(Gra)
    E = [[a for a in Set(e)] for e in Array(E)]
    E = pm.to_one_based_indexing(E)
    return graph_from_edges(E)
end

function edgesDualGraph(subd)
    dg = subdDualGraph(subd)
    dg_edges = collect(edges(dg))
    maxCells = maximal_cells(subd)
    dualGraphEdgesAsVertices = [intersect(maxCells[src(e)], maxCells[dst(e)]) for e in dg_edges]
    return dualGraphEdgesAsVertices
end


function subd2CodimLeq1Matroids(subd, d, n)
    maxMatroids = [plueckerList2Matroid(Vector{Int64}(pm.@convert_to Vector r), d, n) for r in maximal_cells(subd) ]    
    codim1Matroids = [plueckerList2Matroid(Vector{Int64}(pm.@convert_to Vector r), d, n) for r in edgesDualGraph(subd) ]  
    return (maxMatroids, codim1Matroids) 
end

function subdMatroidsNonLeaves(subd, d, n)
    Mats = subd2Matroids(subd, d, n)
    dg = subdDualGraph(subd)
    Leaves = leavesGraph(dg)
    notLeaves = [v for v in 1:n_vertices(dg) if !(v in Leaves)]
    return Mats[notLeaves]
end

function commonBasis(Ms)
    return intersect([bases(M) for M in Ms]...)
end

function subdMatroidsLeaves(subd, d, n)
    Mats = subd2Matroids(subd, d, n)
    dg = subdDualGraph(subd)
    Leaves = leavesGraph(dg)
    return Mats[Leaves]
end

function leavesGraph(dg)
    return [v for v in 1:n_vertices(dg) if length(all_neighbors(dg, v)) == 1 ]
end

function leafEdgesDualGraphAsCells(subd)
    dg = subdDualGraph(subd)
    edges_dg = collect(edges(dg))
    lG = leavesGraph(dg)
    leafEdges = [e for e in edges_dg if (src(e) in lG || dst(e) in lG )]
    maxCells = maximal_cells(subd)
    dualGraphEdgesAsVertices = [intersect(maxCells[src(e)], maxCells[dst(e)]) for e in leafEdges]
    return dualGraphEdgesAsVertices
end


function subd2LeavesAndCenter(subd, d, n)
    leafMaxMatroids = subdMatroidsLeaves(subd, d, n)
    leafEdgeMaxMatroids = [plueckerList2Matroid(Vector{Int64}(pm.@convert_to Vector r), d, n) for r in leafEdgesDualGraphAsCells(subd)]
    remainingMaxMatroids = subdMatroidsNonLeaves(subd, d, n)
    
    return (leafMaxMatroids, leafEdgeMaxMatroids, remainingMaxMatroids) 
end

## Dimension of thin Schubert cells and checking $B$-maximal

Functions in ```Bmaximal.jl```

---
**Function**: ```dimBRing(M::Matroid, xi::Vector{Int64})```

*Description*: Given a matroid and a basis $\xi$, returns the number of bases $\lambda$ of $M$ such that the symmetric difference $\lambda \Delta \xi$ has 2 elements. 

*Example*: 

```
M = cycle_matroid(Graphs.complete_graph(4))
dimBRing(M, [1,3,5]) == 6.
```
---

**Function**: ```basis2DimB(M::Matroid)```

*Description*: Returns ```D``` a ```Dict{Vector{Int64}, Int64}``` whose keys are the bases of ```M``` and the value on a basis ```xi``` is ```dimBRing(M,xi)```. 

*Example*: 
```
M = cycle_matroid(Graphs.complete_graph(4))
basis2DimB(M) == Dict(
[1, 3, 4] => 7, [1, 3, 6] => 7, [2, 3, 5] => 7, [1, 4, 6] => 7, 
[2, 4, 5] => 7, [1, 2, 5] => 7, [3, 4, 6] => 7, [2, 5, 6] => 7, 
[1, 3, 5] => 6, [4, 5, 6] => 6, [2, 3, 4] => 7, [2, 3, 6] => 6,
[1, 5, 6] => 7, [3, 4, 5] => 7, [1, 2, 4] => 6, [1, 2, 6] => 7
)
```

---
**Function**: ```basis2DimB_Mins(M::Matroid)```

*Description*: Similar to ```basis2DimB```, but the keys are only those bases ```xi``` so that ```dimBRing(M,xi)``` is minimal.  
*Example*: 
```
M = cycle_matroid(Graphs.complete_graph(4))
basis2DimB_Mins(M) == Dict([1, 3, 5] => 6, [4, 5, 6] => 6, [2, 3, 6] => 6, [1, 2, 4] => 6)

```

---
**Function**: ```dimTSC(M::Matroid,  F::Ring, xi::Vector{Int64}, R::MPolyRing_dec, x::fmpz_mpoly)```

*Description*: Computes the dimension of the thin Schubert cell $\operatorname{Gr}(M)$ of the matroid $M$ defined over the ring ```F```; here ```xi``` can be any basis of ```M```. 

*Example*: 
```
M = cycle_matroid(Graphs.complete_graph(4))
R, x = makePolyRing(3,6,QQ)
dimTSC(M, QQ, [1,3,5], R, x) == 5

```
---
**Function**: ```dimTSC_Optimized(M::Matroid,  F::Ring, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: Computes the dimension of the thin Schubert cell $\operatorname{Gr}(M)$ of the matroid $M$ defined over the ring ```F```, but it finds a basis ```xi```  of ```M``` that yields a simpler expression for the coordinate ring. 

*Example*: 
```
M = cycle_matroid(Graphs.complete_graph(4))
R, x = makePolyRing(3,6,QQ)
dimTSC_Optimized(M, QQ, R, x) == 5

```
---
**Function**: ```isBMaximal(M::Matroid,  F::Ring, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: Returns ```true``` if the matroid ```M``` is $B$-maximal, and ```false``` otherwise. 

*Example*: 
```
M1 = cycle_matroid(Graphs.complete_graph(4))
isBMaximal(M1,  QQ) == false

M2 = matroid_from_nonbases([[1,2,3], [1,4,5], [2,4,6]], 6) 
isBMaximal(M2,  QQ) == true

```

---
**Function**: ```dimsTSC_Ms(Ms::Vector{Matroid}, F::Ring, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: Returns a ```Dict``` whose keys are elements of ```Ms``` and the value at a matroid is the dimension if its thin Schubert cell. There should be a basis common to each matroid in ```Ms```. 

*Example*: 
```
R, x = makePolyRing(3,8,QQ)
Delta38 = Polymake.polytope.hypersimplex(3,8);
vDelta38 = Matrix{Int64}(Delta38.VERTICES);
w = [0,0,0,0,1, 1,1,1,0,1, 0,1,0,1,0, 0,1,0,1,0, 1,1,0,1,0, 0,0,1,0,0, 0,0,0,0,0, 1,1,1,1,1, 0,1,0,0,0, 0,0,1,0,0, 0,0,0,0,0,0]
subd = SubdivisionOfPoints(vDelta38[:, 2:9], w)
Ms = subd2Matroids(subd, 3, 8)
D = dimsTSC_Ms(Ms, QQ, R, x)
all([D[Ms[1]] == 8, D[Ms[2]] == 8, D[Ms[3]] == 8, D[Ms[4]] == 9, D[Ms[5]] == 8, D[Ms[6]] == 8 ])
```

---
**Function**: ```checkBMaximal(Ms::Vector{Matroid}, dimMs::Dict{Matroid, Int64}, F::Ring, xi::Vector{Int64})```

*Description*:  Returns ```true``` if each matroid in ```Ms``` is $(B,\xi)$-maximal. 

*Example*: 
```
R, x = makePolyRing(3,8,QQ)
Delta38 = Polymake.polytope.hypersimplex(3,8);
vDelta38 = Matrix{Int64}(Delta38.VERTICES);
w = [0,0,0,0,1, 1,1,1,0,1, 0,1,0,1,0, 0,1,0,1,0, 1,1,0,1,0, 0,0,1,0,0, 0,0,0,0,0, 1,1,1,1,1, 0,1,0,0,0, 0,0,1,0,0, 0,0,0,0,0,0]
subd = SubdivisionOfPoints(vDelta38[:, 2:9], w)
Ms = subd2Matroids(subd, 3, 8)[1:3]
D = dimsTSC_Ms(Ms, QQ, R, x)
checkBMaximal(Ms, D, QQ, [1,2,3]) == true

```
---
**Function**: ```existsBasisBMaximal_DimsPrecomputed(Ms, dimMs, F, Bs)```

*Description*: 

*Example*: 
```

```

---
**Function**: ```existsBasisBMaximal_Ms(Ms, F, R, x, Bs)```

*Description*:  

*Example*: 
```

```

---
**Function**: ```isFinBMaximal(Fin, Ms, F, R, x)```

*Description*:  

*Example*: 
```

```

---
**Function**: ```allFinsBMaximal(Fins, Ms, F, R, x)```

*Description*:  

*Example*: 
```

```


In [ ]:
## add documentation

function bases2DimBDifference(M)
    D = basis2DimB(M)
    m = minimum(values(D))
    
    return Dict([b => D[b] - m for b in keys(D)])
end

function optimalBasesForLimit(Ms)
    DMs = Dict([M => bases2DimBDifference(M) for M in Ms])
    basesOptions = commonBasis(Ms)
    dimsByBasis = Dict([ b=> sum([DMs[M][b] for M in Ms]) for b in basesOptions])
    
    minDimSum = minimum(values(dimsByBasis))
    
    minDimsByBasis = [b for b in keys(dimsByBasis) if  dimsByBasis[b] == minDimSum ]
    return minDimsByBasis
end


function dimBRing_MatList(Ms, xi)
    basesMs = []
    for M in Ms
        append!(basesMs, bases(M))
    end
    unique!(basesMs)
    VarB = [a for a in basesMs if length(intersect(a,xi)) == 2  ] 
    return length(VarB)
end


function dimLimitTSC(Ms, F, R, x)

    optB = optimalBasesForLimit(Ms)
    stratumMs = limitTSC(Ms, F, first(optB), R, x)
    gensSMs = minimal_generating_set(stratumMs)
  
    expM = exponentVecs(gensSMs, R, x)
    dBMs = dimBRing_MatList(Ms, first(optB))

    if searchUniUpperTriangular(expM)
        result = dBMs - length(gensSMs)
    else
        result = dBMs - codim(stratumMs)
    end
    
    return result
end


In [ ]:
function dimBRing(M, xi)
    basesM = bases(M)
    VarB = [a for a in basesM if length(intersect(a,xi)) == 2  ] 
    return length(VarB)
end

function basis2DimB(M)
    dimBDict = Dict{Vector{Int64}, Int64}([b => dimBRing(M,b) for b in bases(M)])    
    return dimBDict
end

function basis2DimB_Mins(M)
    DM = basis2DimB(M)
    minB = min(values(DM)...)
    return Dict{Vector{Int64}, Int64}([b => DM[b] for b in keys(DM) if DM[b] == minB])  
end


function dimTSC(M, F, xi, R, x)
    stratumM = TSC(M, F, xi, R, x)
    dBM = dimBRing(M, xi)
    return dBM - codim(stratumM)
end


function dimTSC_Optimized(M, F, R, x)
    BDims = basis2DimB_Mins(M)
    xi = first(keys(BDims))
    
    stratumM = TSC(M, F, xi, R, x)
    dBM = dimBRing(M, xi)
    return dBM - codim(stratumM)
end

function isBMaximal(M, F, R, x)
    DM = basis2DimB_Mins(M)
    xi = first(keys(DM))
    minB = DM[xi]
    dimStratum = dimTSC(M,F,xi,R,x)

    return dimStratum == minB
end

function dimsTSC_Ms(Ms, F, R, x)
    return Dict([M => dimTSC_Optimized(M, F, R, x) for M in Ms])
end

function checkBMaximal(Ms, dimMs, F, xi)
    #dimsTSC_Ms = dimsTSC_Ms(Ms, F)
    dimsBRing_Ms = Dict([M => dimBRing(M, xi) for M in Ms])
    return all([dimMs[M] == dimsBRing_Ms[M] for M in Ms])
end

function existsBasisBMaximal_DimsPrecomputed(Ms, dimMs, F, Bs)
    return [xi for xi in Bs if checkBMaximal(Ms, dimMs, F, xi)]
end
    
function existsBasisBMaximal_Ms(Ms, F, R, x, Bs)
    dimMs = dimsTSC_Ms(Ms, F, R, x)
    return existsBasisBMaximal_DimsPrecomputed(Ms, dimMs, F, Bs)
end


function isFinBMaximal(Fin, Ms, F, R, x)
    MsInFin = Dict([i=>Ms[i] for i in Fin[1]])
    MsExposedFin = Dict([i=>Ms[i] for i in Fin[1] if i ∉ Fin[2]])
    Bs = commonBasis(values(MsInFin))
    return existsBasisBMaximal_Ms(values(MsExposedFin), F, R, x, Bs)    
end

function allFinsBMaximal(Fins, Ms, F, R, x)
    return all([length(isFinBMaximal(Fin, Ms, F, R, x)) > 0 for Fin in Fins])
end


## Simplifying ideals

Functions in `simplifyingIdeal.jl`

---
**Function**: `supportPolynomial(f::fmpq_mpoly, R, x)`

*Description*: Returns the sum of the exponent vectors of `f`, which is an element of the ring `R`. 

*Example*: 
```
R, x = makePolyRing(2,5,QQ)
f = 2*x[1,1]*x[2,3] - 4x[1,2]^3
supportPolynomial(f,R,x) == [1, 0, 3, 0, 0, 1]

```
---
**Function**: `supportSize(v::Vector{Int64})`

*Description*: Returns the number of nonzero entries of `v`. 

*Example*: `supportSize([1, 0, -2, 0, 0, 3]) == 3`


---
**Function**: `isUnitVector(v::Vector{Int64})`

*Description*: Returns `true` if `v` has exactly one nonzero element, which is 1; returns `false` otherwise

*Example*: 

```
isUnitVector([1, 0, -2, 0, 0, 3]) == false
isUnitVector([0, 0, 0, 1, 0]) == true

```
---
**Function**: `addColumnStillUniUpperTriangular(nr::Int64, nc::Int64, pivotsM::Vector{Int64}, optionsM::Vector{Vector{Int64}})`

*Description*: For each vector `v` in `optionsM`, remove from `v` the rows whose index is in `pivotsM`. If the new vector is a unit vector, let `newPivotsM` be `pivotsM` union `[i]` and  `newOptionsM` be `optionsM` \ `[v]`.   This function returns a `Vector` of all such pairs `(newPivotsM::Vector{Int64}, newOptionsM::Vector{Vector{Int64}})`. 


*Example*: 

---
**Function**: `removeZeroRows(M::Matrix{Int64})`

*Description*: Returns a new matrix obtained by removing all rows of `M` that are identically 0. 

*Example*: 

```
M = [0 0 0; 1 0 0; 0 0 0; 1 -2 3]
removeZeroRows(M) == [1 0 0; 1 -2 3]
```
---
**Function**: `searchUniUpperTriangular(M::Matrix{Int64})`

*Description*: Returns `true` if `M` has an full-row unimodular upper-triangular submatrix, after possibly permuting the rows and columns. It returns `false` otherwise.  

*Example*: 


---
**Function**: `exponentVecs(Igens::Vector{fmpq_mpoly}, R::MPolyRing_dec, x::fmpz_mpoly)`

*Description*: Returns a `Matrix{Int64}` whose rows are the vectors `supportPolynomial(f,R,x)` for `f` in `Igens`. 

*Example*: 
```
R, x = makePolyRing(2,5,QQ)
I = ideal([2*x[1,1]*x[2,3] - 4*x[1,2]^3, x[1,2]*x[2,3] - 3*x[1,2]*x[2,2]])
exponentVecs(gens(I), R, x) == Matrix{Int64}([1 0 3 0 0 1; 0 0 2 1 0 1])
```

---
**Function**: `systemHasUniUpperTriangle(Igens::Vector{fmpq_mpoly}, R::MPolyRing_dec, x::fmpz_mpoly)`

*Description*: Let `M` be the matrix obtained from `exponentVecs(Igens, R, x)` by removing the rows identically equal to 0 (apply `removeZeroRows`).  This function returns `searchUniUpperTriangular(M::Matrix{Int64})`.

*Example*: 
```
R, x = makePolyRing(2,5,QQ)
I = ideal([R(0), 2*x[1,1]*x[2,3] - 4*x[1,2]^3, x[1,2]*x[2,3] - 3*x[1,2]*x[2,2]])
J = ideal([2*x[1,1]^2 - 4*x[1,2]^3, x[1,2]^2 - 3*x[2,2]^2])
systemHasUniUpperTriangle(gens(I), R, x) == true
systemHasUniUpperTriangle(gens(J), R, x) == false
```

---
**Function**: `localizingSemiGroup(Ms::Vector{Matroid}, F::Ring, B::Vector{Int64}, R::MPolyRing_dec, x::fmpz_mpoly)`

*Description*: 

*Example*: 
```
R, x = makePolyRing(2,5,QQ)
Delta25 = Polymake.polytope.hypersimplex(2,5);
vDelta25 = Matrix{Int64}(Delta25.VERTICES[1:10,1:6]);
subd = subdivision_of_points(vDelta25[:, 2:6], [1,0,0,0,0,0,0,0,0,1])
Ms = subd2Matroids(subd, 2, 5)

S = localizingSemiGroup(Ms, QQ, [1,4], R, x)
S2gens = [x[1,1],x[1,2],x[1,3],x[2,1],x[2,2],x[2,3]]
S2 = MPolyPowersOfElement(S2gens[1])
for i in 2:6
    S2 = product(S2,  MPolyPowersOfElement(S2gens[i]))
end
S == S2
```

---
**Function**: `localizeLimitRing(Ms::Vector{Matroid}, F::Ring, B::Vector{Int64}, R::MPolyRing_dec, x::fmpz_mpoly)`

*Description*: Returns the ring `R` localized at `S = localizingSemiGroup(Ms, F, B, R)`. 

*Example*: 

---
**Function**: `coefficientPoly(v::fmpq_mpoly, f::fmpq_mpoly, R::MPolyRing_dec)`

*Description*: If the variable `v` has degree 1 in `f`, returns its coefficient. Otherwise, returns `"can't isolate"`.

*Example*: 

---
**Function**: `varCanBeEliminated(v::fmpq_mpoly, f::fmpq_mpoly, R::MPolyRing_dec, S::MPolyPowersOfElement)`

*Description*: If the variable `v` has degree 1 in `f` and its coefficient is a unit in $S^{-1}R$, returns `true`, otherwise, returns `false`.

*Example*: 

---
**Function**: `solve4v(v::fmpq_mpoly, f::fmpq_mpoly, R::MPolyRing_dec, S::MPolyPowersOfElement, SR::MPolyLocalizedRing)`

*Description*: If the variable `v` can be solved for using the equation `f`, returns the solution. 

*Example*: 

---
**Function**: `idealEliminateVariable(Igens::Vector{fmpq_mpoly}, R::MPolyRing_dec, x::fmpz_mpoly, S::MPolyPowersOfElement, SR::MPolyLocalizedRing, yi::Int64, yj::Int64, fy)`

*Description*: Set `v = x[yi, yj]`, and suppose `v` can be solved for using the polynomial `f` in `Igens`, and `v = fy`. This function returns the ideal obtained by performing this substitution. 

*Example*: 

---
**Function**: `canReduceIdeal(Igens::Vector{fmpq_mpoly}, R::MPolyRing_dec, x::fmpz_mpoly, S::MPolyPowersOfElement, SR::MPolyLocalizedRing)`

*Description*: This function systematically eliminates variables using `idealEliminateVariable`. If the result is the 0-ideal, returns `true`, else, returns `false`. 

*Example*: 



In [ ]:
function supportPolynomial(f, R, x)
    expVecs = [exponent_vector(f,i) for i in 1:length(f)]
    if length(expVecs) == 0
        return zeros(Int64,  length(x))
    end
    return sum(expVecs)
end

function supportSize(v)
    return length([a for a in v if a ≠ 0])
end

function isUnitVector(v)
    n = length(v)    
    Z = [i for i in 1:n if v[i] == Int64(0)]
    O = [i for i in 1:n if v[i] == Int64(1)]
    return (length(Z) == n-1 && length(O) == 1)
end

#UUT = Unimodular Upper triangular
# colPivot = Pair{Column, Pivot}

function addColumnStillUniUpperTriangular(nr, nc, pivotsM, optionsM)
    outp = []    
    notPivots = sort!([i for i in 1:nr if i ∉ pivotsM]); #print(notPivots)
    for cindex in 1:length(optionsM)
        c = optionsM[cindex]
        cNonPivots = c[notPivots]
        #print(cNonPivots, "\n")
        if isUnitVector(cNonPivots)
            position1 = findall(x->x==1, cNonPivots)[1]
            #print("position1 = ", position1, "\n")
            newPivotsM = sort!(vcat(pivotsM, notPivots[position1]))
            #print("newPivots = ", newPivotsM, "\n")
            newOptionsM = [optionsM[i] for i in 1:length(optionsM) if i ≠ cindex]
            push!(outp, (newPivotsM, newOptionsM) )
        else
            continue
        end
    end
    return outp    
end

function removeZeroRows(M)
    nr, nc = size(M)
    rowsM = [M[i,:] for i in 1:nr]
    nonZeroRows = [v for v in rowsM if supportSize(v) > 0]
    return Matrix{Int64}([nonZeroRows[i][j] for i in 1:length(nonZeroRows), j in 1:nc])
end

function searchUniUpperTriangular(M)
    Mno0 = removeZeroRows(M) # to define
    nr, nc = size(Mno0); #print("nr=", nr, " nc=", nc, "\n")
    
    if nr == 0
        return true
    end
    
    if nr == 1
        return any([a == Int64(1) for a in Mno0]) || all([a == Int64(0) for a in Mno0])
    end
    
    initialPivots = Vector{Int64}([])
    colsMno0 = [Mno0[:,i] for i in 1:nc]
    
    Stepa = addColumnStillUniUpperTriangular(nr, nc, initialPivots, colsMno0)
    #print("Stepa = ",Stepa, "\n")
    
    for i in 1:(nr-1)
        Stepb = []
        for i in 1:length(Stepa)
            addData = addColumnStillUniUpperTriangular(nr, nc, Stepa[i][1], Stepa[i][2])
            append!(Stepb, addData)
        end 
        Stepa = unique!(Stepb)
    end
    return length(Stepa) > 0
end

function exponentVecs(Igens, R, x)
    rowsEV = [supportPolynomial(f, R, x) for f in Igens]
    return Matrix{Int64}([rowsEV[i][j] for i in 1:length(rowsEV), j in 1:length(x)] )
end

function systemHasUniUpperTriangle(Igens, R, x)
    expVecs = exponentVecs(Igens, R, x)
    return searchUniUpperTriangular(expVecs)
end


function localizingSemiGroup(Ms, F, B, R, x)
    basesX = reduce(append!, map(M->projectedBases(M, F, B, R, x), Ms))
    return reduce(product, map(powers_of_element, basesX)) 
end


function localizeLimitRing(Ms, F, B, R, x)
    sTotal = localizingSemiGroup(Ms, F, B, R, x)
    return localization(sTotal)[1]    
end

function coefficientPoly(v, f, R)
    terms_v = [term(f,i) for i in 1:length(f) if v in vars(monomial(f,i))]
    #print("terms_v = ", terms_v)
    if length(terms_v) == 0
        return R(0)
    else
        for t_v in terms_v
            dict_tv = Dict([pair for pair in factor(t_v)]) 
            if dict_tv[v] > 1
                return "can't isolate"
            else
                continue
            end
        end
    end
    
    fv = sum(terms_v)
    fvFactor = factor(fv)
    ufv = unit(fvFactor)
    fvFactorDict = Dict([pair for pair in fvFactor]) 
    coef_v = prod([k^(fvFactorDict[k]) for k in keys(fvFactorDict) if k ≠ v ])
    return ufv * coef_v
    
end

function varCanBeEliminated(v, f, R, S)
    cv = coefficientPoly(v, f, R)
    if typeof(cv) == String 
        if cv == "can't isolate"
            return false
        end
    end 
    return cv in S
end


function solve4v(v, f, R, S, SR)
    if !varCanBeEliminated(v,f,R,S)
        return "can't solve"
    end
    cv = coefficientPoly(v, f, R)
    terms_notv = [term(f,i) for i in 1:length(f) if v ∉ vars(monomial(f,i))]
    if length(terms_notv) == 0
        return "can't solve"
    end
    return R(-1) * sum(terms_notv) // cv
end

function idealEliminateVariable(Igens, R, x, S, SR, yi, yj, fy)
    nr_x , nc_x = size(x); #print("nr_x = ", nr_x, "\n")
    nrnc_x = nr_x*nc_x; #print("nrnc_x = ", nrnc_x, "\n")
    Sx = [SR(x[i,j]) for i in 1:nr_x, j in 1:nc_x]; #print("Sx = ", Sx, "\n"); print("Sxij type = ", typeof(Sx[yi,yj]))
    Sx[yi,yj] = SR(fy); 
    
    #print("fy = ", fy, "\n")
    
    F = hom(R, SR, z->z, reshape(Sx, nrnc_x))
    
    newIgens = []
    for g in Igens
        #print("g = ", g, "\n")
        Fg = numerator(F(g))
        if Fg == 0
            continue
        end
        factoredFg = factor(Fg)
        factoredFgDict = Dict([pair for pair in factoredFg])
        newFactors = Dict()
        for h in keys(factoredFgDict)
            if h ∉ S
                newFactors[h] = factoredFgDict[h]
            end
        end
        new_g = prod([k^(newFactors[k]) for k in keys(newFactors)])
        push!(newIgens, new_g)        
    end
    
    J = ideal(R, newIgens)
    
    # update S
    
    return (unique!(gens(J)), R, x, S, SR)
end

function canReduceIdeal(Igens, R, x, S, SR)
    if systemHasUniUpperTriangle(Igens, R, x)
        return true
    end
    
    stop = false
    
    for f in Igens
        
        #print("f = ", f, "\n")
        
        if stop
            continue
        end
        
        for v in vars(f)
            #print("v = ", v, "\n")
            
            solve_v = solve4v(v, f, R, S, SR)
            #print("solve_v = ", solve_v, "\n")
            
            if typeof(solve_v) == String
                if solve_v == "can't solve"
                    continue
                end
            else
                v_index = findall(t->t == v, x)[1]
                v_r = v_index[1]; #print("v_r = ", v_r, "\n")
                v_c = v_index[2]; #print("v_c = ", v_c, "\n")
                elim_v = idealEliminateVariable(Igens, R, x, S, SR, v_r, v_c, solve_v)
                #print("elim = ", elim_v[1], "\n")
                stop = true
                
                return canReduceIdeal(elim_v[1], R, x, S, SR)
            end
        end      
    end
    
    if !stop
        return false
    end
    
end

## Specific Matroids

**Function**: `matroidU(l1::Vector{Int64}, l2::Vector{Int64}, l3::Vector{Int64}, l4::Vector{Int64})`

*Description*: Creates the matroid $\mathsf{U}$ whose rank-1 flats are `l1,l2,l3,l4`. 


*Example*: 

```
U = matroidU([1,2],[3,4],[5,6],[7,8])
hyperplanes(U) == [[1, 2, 3, 4], [1, 2, 5, 6], [1, 2, 7, 8], [3, 4, 5, 6], [3, 4, 7, 8], [5, 6, 7, 8]]
```
---

**Function**: `matroidV(l1::Vector{Int64}, l2::Vector{Int64}, l3::Vector{Int64}, l4::Vector{Int64}, l5::Vector{Int64})`

*Description*: Creates the matroid $\mathsf{U}$ whose rank-1 flats are `l1,l2,l3,l4`. 


*Example*: 

```
V = matroidV([1,2],[3,4],[5,6],[7],[8])
hyperplanes(V) == [[1, 2, 3, 4, 5, 6], [7, 8], [1, 2, 7], [1, 2, 8], [3, 4, 7], [3, 4, 8], [5, 6, 7], [5, 6, 8]]
```
---

**Function**: `matroidW(l1::Vector{Int64}, l2::Vector{Int64}, l3::Vector{Int64}, l4::Vector{Int64}, l5::Vector{Int64})`

*Description*: Creates the matroid $\mathsf{U}$ whose rank-1 flats are `l1,l2,l3,l4`. 


*Example*: 

```
W = matroidW([1,2],[3,4],[5,6],[7],[8])
hyperplanes(W) == [[1, 2, 3, 4, 5, 6], [1, 2, 7, 8], [3, 4, 7], [3, 4, 8], [5, 6, 7], [5, 6, 8]]
```


In [ ]:
function matroidU(l1, l2, l3, l4)
    ls = [l1, l2, l3, l4]
    hyp = [vcat(l1,l2), vcat(l1,l3), vcat(l1,l4), vcat(l2,l3), vcat(l2,l4), vcat(l3,l4)]
    n_M = sum([length(l) for l in ls])
    return matroid_from_hyperplanes(hyp, n_M)
end

function matroidV(l1,l2,l3,l4,l5)
    ls = [l1,l2,l3,l4,l5]
    hyp = [vcat(l1,l2,l3), vcat(l4,l5), vcat(l1,l4), vcat(l1,l5), vcat(l2,l4), vcat(l2,l5), vcat(l3,l4), vcat(l3,l5)]
    n_M = sum([length(l) for l in ls])
    return matroid_from_hyperplanes(hyp, n_M)
end


function matroidW(l1,l2,l3,l4,l5)
    ls = [l1,l2,l3,l4,l5]
    hyp = [vcat(l1,l2,l3), vcat(l1,l4,l5), vcat(l2,l4), vcat(l2,l5), vcat(l3,l4), vcat(l3,l5)]
    n_M = sum([length(l) for l in ls])
    return matroid_from_hyperplanes(hyp, n_M)
end